# 第三十讲：谱聚类（Spectral Clustering）
___

## 1. 复杂网络（complex network）

复杂网络可以表示为$\mathbf{G = (V, E)}$，其中$\mathbf{V = \{v_1, v_2, \ldots, v_n\}}$表示所有的节点的集合，$\mathbf{E = \{(v_i, v_j) \lvert v_i, v_j \in V\}}$。复杂网络数据结构表示的时候，可用邻接矩阵或者链表的序列表示。

复杂网络可以是有向图（directed graph）、无向图（undirected graph）、无权图（unweighted graph）、加权图（weighted graph）或者超图（hypergraph）。

## 2. 网络聚团结构（cluster structure）

也称网络社团结构（network community structure），是复杂网络中最普遍和最重要的拓扑属性（topological properties）之一。网络簇是整个网络中的稠密连接分支（densely connected subnetwork），具有同簇内部节点之间连接密集（densely），相反，不同簇节点之间连接稀疏（sparsely）的特征。

## 3. 复杂网络分类

随机网络（random network）、小世界网络（small-world network）、无标度网络（scale-free network）。

## 4. 图的拉普拉斯矩阵（Laplacian matrix）

也称为基尔霍夫矩阵（Kirchhoff matrix），是图的一种矩阵表示形式。

> 对于一个有n个节点的图G=(V,E)，其Laplacian矩阵定义为：$\mathbf{L = D - W}$

其中，$\mathbf{D}$为图的度矩阵，$\mathbf{W}$为图的邻接矩阵，为了方便计算，通常我们设$\mathbf{w_{ii}} = 0$。

对于一个定点数为$n$的邻接矩阵来说，可以写成：
$$
\mathbf{W} = \left(\begin{array}{ccc}
w_{11} & \cdots & w_{1n}\\
\vdots & \vdots & \vdots\\
w_{n1} & \cdots & w_{nn}
\end{array}
\right)
$$

度矩阵$\mathbf{D}$为
$$
\mathbf{D} = \left(
\begin{array}{cccc}
d_1 & 0 & \cdots & 0\\
0 & d_2 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots\\
0 & 0 & \cdots & d_n
\end{array}
\right)
$$

其中$d_i = \sum_{j=1}^n w_{ij}$

## 5. Laplacian矩阵的性质

- $\mathbf{L}$是对称半正定矩阵；

- $\mathbf{L}$的最小特征值是0，其对应的特征向量是$\mathbf{1}$；

- $\mathbf{L}$有$n$个非负实特征值$0 \le \lambda_1 \le \lambda_2 \le \cdots \le \lambda_n$，且对于任何实向量$f$，具有
$$
x^T L x = \frac{1}{2}\sum_{i,j=1}^n w_{ij}(x_i - x_j)^2
$$

__证明__:
$$
\begin{array}{lcl}
x^T L x &=& x^T(D-W)x\\
&=& x^T D x - x^TWx\\
&=& \sum_{i=1}^n d_i x_i^2 - \sum_{i,j=1}^n w_{ij}x_i x_j\\
&=& \frac{1}{2}\left(\sum_{i=1}^n d_i x_i^2 - 2\sum_{i,j=1}^n w_{ij}x_ix_j + \sum_{j=1}^n d_j x_j^2 \right)\\
&=& \frac{1}{2}\sum_{i,j=1}^n w_{ij}(x_i - x_j)^2
\end{array}
$$

在某些情况下，拉普拉斯矩阵还可以表示为规范化的形式：
$$
\mathbf{L = D^{-1/2}(D-W)D^{-1/2} = I - D^{-1/2}W D^{-1/2}}
$$

或者随机游走（random walk）的形式：
$$
\mathbf{L = D^{-1}(D-A) = I - D^{-1}W}
$$

## 6. 谱聚类（spectral clustering）

在谱聚类中定义了“切割（cut）”函数的概念，当一个网络被划分成为两个子网络时，“切割”即指子网间的连接密度。谱聚类的目的就是要找到一种合理的分割，使得分割后形成若干子图（subgraph），连接不同的子图的边的权重尽可能低，即“切割”最小，相反同子图内的边的权重则尽可能要高。

### 6.1 目标函数1：$\operatorname{cut}$

“切割”表示的是子网间的密度，即边比较少，且权重比较小。以二分为例，将图聚类成两个类：$S$类和$T$类。假设用$\operatorname{cut}(S,T)$来表示图的划分，我们需要的结果是：
$$
\min\; \operatorname{cut}(S, T) = \sum_{i\in S; j\in T} e_{ij} = W(S, T)
$$
其中$W(S,T)$表示类别$S$与$T$之间边的权重。

如果存在$k$个不同的类别$A_1, A_2, \ldots, A_k$，则最小化的目标函数为：
$$
\min\; \operatorname{cut}(A_1, \ldots, A_k) = \sum_{i=1}^k W(A_i, \bar{A}_i)
$$

但这种“切割”函数存在一定的弊端，也就是容易出现单一的节点作为聚类的一个问题。

### 6.2 其他目标函数：$\operatorname{RatioCut}$和$\operatorname{NCut}$

为了能够让每个类都有合理的大小，目标函数中应该使得$A_1, A_2, \ldots, A_k$足够大，提出了$\operatorname{RatioCut}(A_1, \ldots, A_k)$和$\operatorname{NCut}(A_1, \ldots, A_k)$函数：

- $\operatorname{RatioCut}(A_1, \ldots, A_k) = \frac{1}{2}\sum_{i=1}^k \frac{W(A_i, \bar{A}_i)}{\lvert A_i \rvert}$

- $\operatorname{RatioCut}(A_1, \ldots, A_k) = \frac{1}{2}\sum_{i=1}^k \frac{W(A_i, \bar{A}_i)}{\operatorname{vol}(A_i)}$

其中$\lvert A_i \rvert$表示$A_i$类中包含的节点的数量，而$\operatorname{vol}(A_i)$表示的是类$A_i$内连接总度数的大小。

### 6.3 谱聚类的优化目标函数与拉普拉斯矩阵的关系

对于二分类$k=2$的聚类问题，其优化目标函数为：
$$
\min\; \operatorname{RatioCut}(A, \bar{A})
$$

首先，定义向量$x = (x_1, \ldots, x_n)^T$，且满足：
$$
x_i = \left\{
\begin{array}{ll}
\sqrt{\frac{\lvert \bar{A} \rvert}{\lvert A \rvert}}, & v_i \in A\\
\sqrt{\frac{\lvert A \rvert}{\lvert \bar{A} \rvert}}, & v_i \in \bar{A}\\
\end{array}
\right.
$$

已知
$$
\begin{array}{lcl}
x^T L x &=& \frac{1}{2}\sum_{i,j=1}^n w_{ij}(x_i - x_j)^2 \\
&=& \frac{1}{2}\sum_{i \in A, j\in \bar{A}} w_{ij} \left( \sqrt{\frac{\lvert \bar{A} \rvert}{\lvert A \rvert}} + \sqrt{\frac{\lvert A \rvert}{\lvert \bar{A} \rvert}}\right)^2 + \frac{1}{2}\sum_{i \in \bar{A}, j\in A} w_{ij} \left( -\sqrt{\frac{\lvert A \rvert}{\lvert \bar{A} \rvert}} - \sqrt{\frac{\lvert \bar{A} \rvert}{\lvert A \rvert}}\right)^2\\
&=& \frac{1}{2}\sum_{i \in A, j\in \bar{A}}w_{ij}\left(\frac{\bar{\lvert A \rvert}}{\lvert A \rvert} + 2 + \frac{\lvert A \rvert}{\lvert \bar{A} \rvert}\right) + \frac{1}{2}\sum_{i \in \bar{A}, j\in A}w_{ij}\left(\frac{\lvert \bar{A} \rvert}{\lvert A \rvert} + 2 + \frac{\lvert A \rvert}{\lvert \bar{A} \rvert}\right)\\
&=& \frac{1}{2}\left[\sum_{i \in A, j\in \bar{A}}w_{ij} + \sum_{i \in \bar{A}, j\in A}w_{ij} \right]\left( \frac{\bar{\lvert A \rvert}}{\lvert A \rvert} + 2 + \frac{\lvert A \rvert}{\lvert \bar{A} \rvert} \right)\\
&=& \operatorname{cut}(A, \bar{A})\left( \frac{\lvert A \rvert + \lvert \bar{A} \rvert}{\lvert A \vert} + \frac{\lvert A \rvert + \lvert \bar{A} \rvert}{\lvert \bar{A} \rvert}\right)\\
&=& (\lvert A \rvert + \lvert \bar{A} \rvert) \operatorname{RatioCut}(A, \bar{A})\\
&=& \lvert V \rvert \cdot \operatorname{RatioCut}(A, \bar{A})
\end{array}
$$

这样我们就在拉普拉斯矩阵和最优化图聚类函数之间建立了联系。

由于$\lvert V \rvert$是常数，因此$\min\; \operatorname{RatioCut}(A, \bar{A})$也就相当于：
$$
\begin{array}{ll}
\min & x^T L x\\
\textit{s.t.} & x \perp \mathbf{1}, \lVert x \rVert = \sqrt{n}
\end{array}
$$

同时还有：
$$
x_i = \left\{
\begin{array}{ll}
\sqrt{\frac{\lvert \bar{A} \rvert}{\lvert A \rvert}}, & v_i \in A\\
\sqrt{\frac{\lvert A \rvert}{\lvert \bar{A} \rvert}}, & v_i \in \bar{A}\\
\end{array}
\right.
$$

假设$\lambda$是拉普拉斯矩阵$L$的特征值，对应的特征向量为$x$，则有
$$
L x = \lambda x
$$
左右两端同时乘以$x^T$，得到
$$
\begin{array}{lcl}
x^T L x &=& x^T \lambda x\\
&=& \lambda n
\end{array}
$$

因此要$\min\; x^T L x$，也就是要求$L$的最小特征值。我们已知$L$的最小特征值为0，由Rayleigh-Ritz定理，需要求$L$的次小特征值以及对应的特征向量$x$，最后根据$x$的符号对对应的节点进行聚类：
$$
v_i \in \left\{
\begin{array}{ll}
A, & x_i \ge 0\\
\bar{A}, & x_i < 0
\end{array}
\right.
$$

#### $\S$为什么称为谱聚类？

对于方阵（square matrix）这种线性算子来说，其所有特征值（eigenalues）被统称为方阵的谱（spectrum），其最大的特征值被称为方阵的谱半径（spectral radius）。对于普通矩阵（非方阵）$A$来说，其特征谱则为$A^TA$的所有特征值。

谱聚类方法本质上是一种基于图论的聚类算法，通过对样本数据的拉普拉斯矩阵的特征向量（本质上是一种降维）进行聚类，从而达到对样本数据进行聚类的目的。

### 6.4 从二分类问题到$k$分类问题

取拉普拉斯矩阵$L$的最小的$k$个特征值对应的$k$个特征向量$x^{(1)}, x^{(2)}, \ldots, x^{(k)}$构建对应的特征矩阵：
$$
\left(
\begin{array}{cccc}
x_1^{(1)} & x_1^{(2)} & \cdots & x_1^{(k)}\\
\vdots & \vdots & \vdots & \vdots\\
x_n^{(1)} & x_n^{(2)} & \cdots & x_n^{(k)}
\end{array}
\right)_{n \times k}
$$

将每行作为一个样本，再采用$k$-means进行聚类。

## 7. 小结

综上所述，谱聚类的一般过程是：

<ol type=1>
    <li>对于给定的图$G=(V,E)$，求图的度矩阵$D$和邻接矩阵$A$，计算图的Laplacian矩阵$L=D-A$；</li>
    <li>对Laplacian矩阵进行特征分解，取其前个特征值对应的特征向量，构成$n \times k$的特征向量矩阵；</li>
    <li>利用$k$-Means聚类算法对上述的的特征向量矩阵进行聚类，每一行代表一个样本点。</li>
</ol>

## 8. 谱聚类的拓展

我们还可以通过相似性矩阵构造拉普拉斯矩阵，也就是相似性矩阵
$$
S = \left(
\begin{array}{cccc}
s_{11} & s_{12} & \cdots & s_{1n}\\
\vdots & \vdots & \vdots & \vdots\\
s_{n1} & s_{n2} & \cdots & s_{nn}
\end{array}
\right)
$$
其中
$$
s_{ij} = \exp\left(-\frac{w_{ij}^2}{\sigma^2} \right)
$$

再利用相似性矩阵$S$构造拉普拉斯矩阵$L = D - S$，其中$D$是相似度矩阵计算得到的度矩阵。

#### 【注意】

使用距离矩阵得到的拉普拉斯矩阵进行谱聚类的时候，必须计算$L$的$k$个最大特征值对应的特征向量，而非最小的$k$个特征值。

如果用相似度矩阵得到的拉普拉斯矩阵进行谱聚类，必须计算计算$L$的$k$个最小特征值对应的特征向量。

我们更倾向于使用相似矩阵进行计算。对于距离矩阵，我们总喜欢将其转化为相似矩阵后再执行谱聚类算法。

#### 如何确定谱聚类的$k$值

一种情况下，$k$值是先验的。否则我们可以通过公式$k^* = \operatorname{argmax_k}\,\lvert \lambda_{k+1} - \lambda_k \rvert$得到$k$值。

In [1]:
from __future__ import division
import scipy.io as scio
from scipy import sparse
from scipy.sparse.linalg.eigen import arpack
from numpy import *

In [3]:
def specClust(data, sigma, k):
    data = sparse.csc_matrix.multiply(data, data)
    data = -data/(2*sigma**2)
    S = sparse.csc_matrix.expm1(data) + \
        sparse.csc_matrix.multiply(sparse.csc_matrix.sign(data), sparse.csc_matrix.sign(data))
    D = S.sum(axis=1)
    D = sqrt(1/D)
    n = len(D)
    D = D.T
    D = sparse.spdiags(D, 0, n, n)
    L = D * S * D
    
    vals, vecs = arpack.eigs(L, k=k, tol=0, which="LM")
    sq_sum = sqrt(multiply(vecs,vecs).sum(axis=1))
    m1, m2 = shape(vecs)
    
    for i in xrange(m1):
        for j in xrange(m2):
            vecs[i,j] = vecs[i,j] / sq_sum[i]
            
    myCentroids, clustAssing = kMeans(vecs, num_clusters)
    
    for i in xrange(shape(clustAssing)[0]):
        print clustAssing[i,0]

In [4]:
def randCent(data, k):
    n = shape(data)[1]
    centroids = matrix(zeros((k, n)))
    for j in range(n):
        minJ = min(data[:,j])
        rangeJ = float(max(data[:,j]), minJ)
        centroids[:,j] = mat(minJ + rangeJ * random.rand(k, 1))
    return centroids

In [5]:
def distEuclidean(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2)))

In [6]:
def kMeans(dataSet, k):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = randCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf; minIndex = ''
            for j in range(k):
                distJI = distEclud(centroids[j,:],dataSet[i,:])
                if distJI > minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex, minDist**2
        
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]
            centroids[cent,:] = mean(ptsInClust, axis=0)
    return centroids, clusterAssment

## 附录1：未正则化的拉普拉斯矩阵的谱聚类算法

1. 输入$n$个数据$\{x_i\}, i=1,\ldots,n$

2. 计算$n \times n$的相似度矩阵$\mathbf{W}$和度矩阵$\mathbf{D}$

3. 计算拉普拉斯矩阵$\mathbf{L = D - W}$

4. 计算$\mathbf{L}$的最小$k$个特征值对应的$k$个特征向量$\mathbf{u_1, \ldots, u_k}$

5. 将$k$个特征向量组成矩阵$\mathbf{U} \in \mathbb{R}^{n \times k}$

6. 执行$k$-means聚类对$\mathbf{U}$进行行聚类。

## 附录2：对称正则化的拉普拉斯矩阵的谱聚类算法

1. 输入$n$个数据$\{x_i\}, i=1,\ldots,n$

2. 计算$n \times n$的相似度矩阵$\mathbf{W}$和度矩阵$\mathbf{D}$

3. 计算拉普拉斯矩阵$\mathbf{L = D^{-1/2}(D - W)D^{-1/2}}$

4. 计算$\mathbf{L}$的最小$k$个特征值对应的$k$个特征向量$\mathbf{u_1, \ldots, u_k}$

5. 将$k$个特征向量组成矩阵$\mathbf{U} \in \mathbb{R}^{n \times k}$

6. 对$\mathbf{U}$的每行进行单位化处理，将其结果存为$\mathbf{Y}$，也就是$\lVert \mathbf{y_i} \rVert = 1$

6. 执行$k$-means聚类对$\mathbf{Y}$进行行聚类。

## 附录3：随机游走的拉普拉斯矩阵的谱聚类算法

1. 输入$n$个数据$\{x_i\}, i=1,\ldots,n$

2. 计算$n \times n$的相似度矩阵$\mathbf{W}$和度矩阵$\mathbf{D}$

3. 计算拉普拉斯矩阵$\mathbf{L = D^{-1}(D - W)}$

4. 计算$\mathbf{L}$的最小$k$个特征值对应的$k$个特征向量$\mathbf{u_1, \ldots, u_k}$

5. 将$k$个特征向量组成矩阵$\mathbf{U} \in \mathbb{R}^{n \times k}$

6. 执行$k$-means聚类对$\mathbf{U}$进行行聚类。

## 附录4： 随机游走与拉普拉斯矩阵的关系

图论中的随机游走是一个随机过程，它从一个节点跳转到另外一个节点。谱聚类即试图找到图的一个划分，使得随机游走在相同的簇中停留而几乎不会游走到其他簇。

#### 随机游走的转移矩阵

从节点$v_i$跳转到节点$v_j$的概率正比于边的权值$w_{ij}$：
$$
p_{ij} = \frac{w_{ij}}{d_i}
$$
这里的$\{p_{ij}\}$构成的矩阵就是拉普拉斯矩阵：
$$
P = D^{-1} W = L_{rw}
$$

对于随机游走算法，可参考文献：

[Fast Random Walk with Restart and Its Applications](http://repository.cmu.edu/cgi/viewcontent.cgi?article=1533&context=compsci)

## 附录5：谱聚类的应用

请参见文章[Spectral clustering of microarray data elucidates the roles of microenvironment remodeling and immune responses in survival of head and neck squamous cell carcinoma.](http://www.ncbi.nlm.nih.gov/pubmed/20458058)